In [70]:
cnt=0
for index, row in events_df.iterrows():
    # Skip rows without PyroCb time information
    if pd.isna(row['pyroCb_time_utc']):
        continue
    if(int(row['pyroCb_time_utc'])>2359):
        continue
    cnt=cnt+1
    if cnt<53:
        continue
    print(row['pyroCb_id'])

202
208
191
212
211
203
161
253
178
264
183
220
252
195
234
219
218
175
168
193
194
192
169
251
173
174
172
171
197


In [64]:
import cdsapi
import wget
c = cdsapi.Client()  # 创建用户
from subprocess import call
import pandas as pd
import os
import datetime

# pyroCb events info
csv_file_path='/Users/liuyangfan/Desktop/Stanford/pyrocast-database/pyrocb_events.csv'
events_df = pd.read_csv(csv_file_path)

# functions
# Function to download with wget
def Downloader(task_url, folder_path, file_name):
    """
    下载器
    :param task_url: 下载任务地址
    :param folder_path: 存放文件夹
    :param file_name: 文件名
    :return:
    """
    wget.download(task_url,out=folder_path+file_name)#folder_path needs '/' in the end

# Function to determine the geographical area
def determine_geographical_area(longitude, latitude):
    # Adjusting the coordinates to the nearest 0.25 degree grid
    north = latitude + 0.25 - (latitude % 0.25)
    south = latitude - (latitude % 0.25)
    if latitude % 0.25==0:
        south=south-0.25
    east = longitude + 0.25 - (longitude % 0.25)
    west = longitude - (longitude % 0.25)
    if longitude % 0.25==0:
        west=west-0.25
    return [north, west, south, east]

    
download_directory = '/Users/liuyangfan/Desktop/Stanford/pyrocast-database/era5/pyroCb_events_/'
vars=['divergence', 'fraction_of_cloud_cover', 'geopotential',
        'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
        'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
        'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
        'vorticity',
    ]
p_levels=[
                '1', '2', '3',
                '5', '7', '10',
                '20', '30', '50',
                '70', '100', '125',
                '150', '175', '200',
                '225', '250', '300',
                '350', '400', '450',
                '500', '550', '600',
                '650', '700', '750',
                '775', '800', '825',
                '850', '875', '900',
                '925', '950', '975',
                '1000',
            ]
cnt=0
for index, row in events_df.iterrows():
    # Skip rows without PyroCb time information
    if pd.isna(row['pyroCb_time_utc']):
        continue
    if(int(row['pyroCb_time_utc'])>2359):
        continue
    cnt=cnt+1
    if cnt<53:
        continue
    print(row['pyroCb_id'])
    # Formatting the event time
    event_time = f"{row['pyroCb_date_utc']} {int(row['pyroCb_time_utc']):04d}"
    event_datetime = datetime.datetime.strptime(event_time, '%Y-%m-%d %H%M')
    # Calculate the time range
    time_before=event_datetime
    time_after=event_datetime + datetime.timedelta(hours=1)
    time_before=datetime.datetime.strftime(time_before,'%Y-%m-%d %H:%M:%S')
    time_after=datetime.datetime.strftime(time_after,'%Y-%m-%d %H:%M:%S')
    # Determine the geographical area
    area = determine_geographical_area(row['pyroCb_longitude'], row['pyroCb_latitude'])
    # Create a directory for the event
    event_dir = os.path.join(download_directory, str(row['pyroCb_id']))
    if not os.path.exists(event_dir):
        os.makedirs(event_dir)

    r=c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': vars,
            'pressure_level': p_levels,
            'year': time_before[:4],#change to loop variable
            'month': time_before[5:7],#change to loop variable
            'day': time_before[8:10],#change to loop variable
            'time': time_before[11:13]+':00',#change to loop variable
            'area': area,#change to loop variable
        },
        )

    url = r.location  # 获取文件下载地址
    path = event_dir+'/'  # 存放文件夹
    filename = 'time_before.nc'  # 文件名
    Downloader(url, path, filename)  # 添加进IDM中下载

    r=c.retrieve(
        'reanalysis-era5-pressure-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': vars,
            'pressure_level': p_levels,
            'year': time_after[:4],#change to loop variable
            'month': time_after[5:7],#change to loop variable
            'day': time_after[8:10],#change to loop variable
            'time': time_after[11:13]+':00',#change to loop variable
            'area': area,#change to loop variable
        },
        )

    url = r.location  # 获取文件下载地址
    path = event_dir+'/'  # 存放文件夹
    filename = 'time_after.nc'  # 文件名
    Downloader(url, path, filename)  # 添加进IDM中下载

2023-11-23 14:54:58,336 INFO Welcome to the CDS
2023-11-23 14:54:58,338 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-11-23 14:54:58,569 INFO Request is completed
2023-11-23 14:55:00,442 INFO Welcome to the CDS
2023-11-23 14:55:00,444 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10396 / 10396

2023-11-23 14:55:00,652 INFO Request is queued
2023-11-23 14:55:01,842 INFO Request is running
2023-11-23 14:55:14,771 INFO Request is completed
2023-11-23 14:55:16,057 INFO Welcome to the CDS
2023-11-23 14:55:16,059 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10392 / 10392

2023-11-23 14:55:16,276 INFO Request is queued
2023-11-23 14:55:17,467 INFO Request is running
2023-11-23 14:55:30,395 INFO Request is completed
2023-11-23 14:55:31,742 INFO Welcome to the CDS
2023-11-23 14:55:31,743 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


...

100% [..............................................................................] 10396 / 10396

2023-11-23 15:42:40,099 INFO Request is queued
2023-11-23 15:42:41,293 INFO Request is running
2023-11-23 15:42:54,234 INFO Request is completed
2023-11-23 15:42:55,324 INFO Welcome to the CDS
2023-11-23 15:42:55,326 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 12772 / 12772

2023-11-23 15:42:55,542 INFO Request is queued
2023-11-23 15:42:56,726 INFO Request is running
2023-11-23 15:43:09,673 INFO Request is completed


100% [..............................................................................] 12768 / 12768

In [68]:
import cdsapi
import wget
c = cdsapi.Client()  # 创建用户
from subprocess import call
import pandas as pd
import os
import datetime

# pyroCb events info
csv_file_path='/Users/liuyangfan/Desktop/Stanford/pyrocast-database/pyrocb_events.csv'
events_df = pd.read_csv(csv_file_path)

# functions
# Function to download with wget
def Downloader(task_url, folder_path, file_name):
    """
    下载器
    :param task_url: 下载任务地址
    :param folder_path: 存放文件夹
    :param file_name: 文件名
    :return:
    """
    wget.download(task_url,out=folder_path+file_name)#folder_path needs '/' in the end

# Function to determine the geographical area
def determine_geographical_area(longitude, latitude):
    # Adjusting the coordinates to the nearest 0.25 degree grid
    north = latitude + 0.25 - (latitude % 0.25)
    south = latitude - (latitude % 0.25)
    if latitude % 0.25==0:
        south=south-0.25
    east = longitude + 0.25 - (longitude % 0.25)
    west = longitude - (longitude % 0.25)
    if longitude % 0.25==0:
        west=west-0.25
    return [north, west, south, east]

    
download_directory = '/Users/liuyangfan/Desktop/Stanford/pyrocast-database/era5/pyroCb_events_pressure_level/'
vars=['divergence', 'fraction_of_cloud_cover', 'geopotential',
        'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
        'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
        'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
        'vorticity',
    ]
p_levels=[
                '1', '2', '3',
                '5', '7', '10',
                '20', '30', '50',
                '70', '100', '125',
                '150', '175', '200',
                '225', '250', '300',
                '350', '400', '450',
                '500', '550', '600',
                '650', '700', '750',
                '775', '800', '825',
                '850', '875', '900',
                '925', '950', '975',
                '1000',
            ]
############### 48 hr before and after ####################
for index, row in events_df.iterrows():
    # Skip rows without PyroCb time information
    if pd.isna(row['pyroCb_time_utc']):
        continue
    if(int(row['pyroCb_time_utc'])>2359):
        continue
    # Formatting the event time
    event_time = f"{row['pyroCb_date_utc']} {int(row['pyroCb_time_utc']):04d}"
    event_datetime = datetime.datetime.strptime(event_time, '%Y-%m-%d %H%M')
    for dt in range(0, 49):
        # Your code here
        print(dt)
        # Calculate the time range
        time_before=event_datetime - datetime.timedelta(hours=dt)
        time_after=event_datetime + datetime.timedelta(hours=1+dt)
        time_before=datetime.datetime.strftime(time_before,'%Y-%m-%d %H:%M:%S')
        time_after=datetime.datetime.strftime(time_after,'%Y-%m-%d %H:%M:%S')
        # Determine the geographical area
        area = determine_geographical_area(row['pyroCb_longitude'], row['pyroCb_latitude'])
        # Create a directory for the event
        event_dir = os.path.join(download_directory, str(row['pyroCb_id']))
        if not os.path.exists(event_dir):
            os.makedirs(event_dir)
    
        r=c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': vars,
                'pressure_level': p_levels,
                'year': time_before[:4],#change to loop variable
                'month': time_before[5:7],#change to loop variable
                'day': time_before[8:10],#change to loop variable
                'time': time_before[11:13]+':00',#change to loop variable
                'area': area,#change to loop variable
            },
            )
    
        url = r.location  # 获取文件下载地址
        path = event_dir+'/'  # 存放文件夹
        filename = 'time_before_'+str(dt)+'.nc'  # 文件名
        Downloader(url, path, filename)  # 添加进IDM中下载
    
        r=c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': vars,
                'pressure_level': p_levels,
                'year': time_after[:4],#change to loop variable
                'month': time_after[5:7],#change to loop variable
                'day': time_after[8:10],#change to loop variable
                'time': time_after[11:13]+':00',#change to loop variable
                'area': area,#change to loop variable
            },
            )
    
        url = r.location  # 获取文件下载地址
        path = event_dir+'/'  # 存放文件夹
        filename = 'time_after_'+str(dt)+'.nc'  # 文件名
        Downloader(url, path, filename)  # 添加进IDM中下载

0


2023-11-24 12:23:44,811 INFO Welcome to the CDS
2023-11-24 12:23:44,812 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-11-24 12:23:45,106 INFO Request is completed
2023-11-24 12:23:46,461 INFO Welcome to the CDS
2023-11-24 12:23:46,462 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10396 / 10396

2023-11-24 12:23:47,824 INFO Welcome to the CDS
2023-11-24 12:23:47,826 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10392 / 103921


...

2023-11-24 12:24:09,550 INFO Welcome to the CDS
2023-11-24 12:24:09,551 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10400 / 104009


2023-11-24 12:24:10,849 INFO Welcome to the CDS
2023-11-24 12:24:10,851 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10396 / 10396

KeyboardInterrupt: 

In [67]:
############### 48 hr before and after ####################
for index, row in events_df.iterrows():
    # Skip rows without PyroCb time information
    if pd.isna(row['pyroCb_time_utc']):
        continue
    if(int(row['pyroCb_time_utc'])>2359):
        continue
    # Formatting the event time
    event_time = f"{row['pyroCb_date_utc']} {int(row['pyroCb_time_utc']):04d}"
    event_datetime = datetime.datetime.strptime(event_time, '%Y-%m-%d %H%M')
    for dt in range(0, 49):
        # Your code here
        print(dt)
        # Calculate the time range
        time_before=event_datetime - datetime.timedelta(hours=dt)
        time_after=event_datetime + datetime.timedelta(hours=1+dt)
        time_before=datetime.datetime.strftime(time_before,'%Y-%m-%d %H:%M:%S')
        time_after=datetime.datetime.strftime(time_after,'%Y-%m-%d %H:%M:%S')
        # Determine the geographical area
        area = determine_geographical_area(row['pyroCb_longitude'], row['pyroCb_latitude'])
        # Create a directory for the event
        event_dir = os.path.join(download_directory, str(row['pyroCb_id']))
        if not os.path.exists(event_dir):
            os.makedirs(event_dir)
    
        r=c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': vars,
                'pressure_level': p_levels,
                'year': time_before[:4],#change to loop variable
                'month': time_before[5:7],#change to loop variable
                'day': time_before[8:10],#change to loop variable
                'time': time_before[11:13]+':00',#change to loop variable
                'area': area,#change to loop variable
            },
            )
    
        url = r.location  # 获取文件下载地址
        path = event_dir+'/'  # 存放文件夹
        filename = 'time_before_'+str(dt)+'.nc'  # 文件名
        Downloader(url, path, filename)  # 添加进IDM中下载
    
        r=c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': vars,
                'pressure_level': p_levels,
                'year': time_after[:4],#change to loop variable
                'month': time_after[5:7],#change to loop variable
                'day': time_after[8:10],#change to loop variable
                'time': time_after[11:13]+':00',#change to loop variable
                'area': area,#change to loop variable
            },
            )
    
        url = r.location  # 获取文件下载地址
        path = event_dir+'/'  # 存放文件夹
        filename = 'time_after_'+str(dt)+'.nc'  # 文件名
        Downloader(url, path, filename)  # 添加进IDM中下载

1


2023-11-24 11:32:11,904 INFO Welcome to the CDS
2023-11-24 11:32:11,905 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-11-24 11:32:12,149 INFO Request is queued
2023-11-24 11:32:13,337 INFO Request is running
2023-11-24 11:32:26,252 INFO Request is completed
2023-11-24 11:32:28,117 INFO Welcome to the CDS
2023-11-24 11:32:28,118 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10392 / 10392

2023-11-24 11:32:28,303 INFO Request is queued
2023-11-24 11:32:29,491 INFO Request is running
2023-11-24 11:32:42,411 INFO Request is completed
2023-11-24 11:32:43,463 INFO Welcome to the CDS
2023-11-24 11:32:43,464 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10392 / 103922


...

100% [..............................................................................] 10396 / 1039630


2023-11-24 12:21:48,190 INFO Request is queued
2023-11-24 12:22:02,836 INFO Request is running
2023-11-24 12:22:22,190 INFO Request is completed
2023-11-24 12:22:23,536 INFO Welcome to the CDS
2023-11-24 12:22:23,538 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


100% [..............................................................................] 10396 / 10396

2023-11-24 12:22:23,791 INFO Request is queued
2023-11-24 12:22:25,029 INFO Request is running


KeyboardInterrupt: 

In [2]:
# generate loop variable dict for pyroCb events
import os
import datetime
import pandas as pd
import cdsapi
from urllib.parse import quote_plus

# Function to determine the geographical area
def determine_geographical_area(longitude, latitude):
    # Adjusting the coordinates to the nearest 0.25 degree grid
    north = latitude + 0.25 - (latitude % 0.25)
    south = latitude - (latitude % 0.25)
    east = longitude + 0.25 - (longitude % 0.25)
    west = longitude - (longitude % 0.25)
    return [north, west, south, east]

# Main function to download ERA5 data
def download_era5_data(csv_file_path, download_directory):
    # Read the CSV file
    events_df = pd.read_csv(csv_file_path)

    # Loop over each event
    for index, row in events_df.iterrows():
        # Skip rows without PyroCb time information
        if pd.isna(row['pyroCb_time_utc']):
            continue

        # Formatting the event time
        event_time = f"{row['pyroCb_date_utc']} {int(row['pyroCb_time_utc']):04d}"

        # Calculate the time range
        time_before, time_after = calculate_time_range(event_time)

        # Determine the geographical area
        area = determine_geographical_area(row['pyroCb_longitude'], row['pyroCb_latitude'])

        # Create a directory for the event
        event_dir = os.path.join(download_directory, str(row['pyroCb_id']))
        if not os.path.exists(event_dir):
            os.makedirs(event_dir)

        # Constructing the API call for each variable
        variables = [
            'divergence', 'fraction_of_cloud_cover', 'geopotential',
            'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
            'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
            'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
            'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
            'vorticity',
        ]

        for var in variables:
            # Constructing the API call
            api_call = f"""
c.retrieve(
    'reanalysis-era5-pressure-levels',
    {{
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': ['{var}'],
        'pressure_level': ['1', '2', '3', '5', '7', '10', '20', '30', '50', '70', '100', '125', '150', '175', '200', '225', '250', '300', '350', '400', '450', '500', '550', '600', '650', '700', '750', '775', '800', '825', '850', '875', '900', '925', '950', '975', '1000'],
        'year': '{event_time[:4]}',
        'month': '{event_time[5:7]}',
        'day': '{event_time[8:10]}',
        'time': ['{time_before}', '{time_after}'],
        'area': {area},
    }},
    '{event_dir}/{quote_plus(var)}.nc')
"""
            # Execute the API call using cdsapi (can be replaced with actual API call execution)
            print(api_call)

# Example usage
download_era5_data(csv_file_path, '/path/to/download/directory')  # Replace with actual download directory path



In [ ]:
pd.

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            'divergence', 'fraction_of_cloud_cover', 'geopotential',
            'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
            'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
            'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
            'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
            'vorticity',
        ],
        'pressure_level': [
            '1', '2', '3',
            '5', '7', '10',
            '20', '30', '50',
            '70', '100', '125',
            '150', '175', '200',
            '225', '250', '300',
            '350', '400', '450',
            '500', '550', '600',
            '650', '700', '750',
            '775', '800', '825',
            '850', '875', '900',
            '925', '950', '975',
            '1000',
        ],
        'year': '2019',#change to loop variable
        'month': '09',#change to loop variable
        'day': '17',#change to loop variable
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],#change to loop variable
        'area': [
            40.25, -130, 40,
            -129.75,
        ],#change to loop variable
    },
    'download.nc')#change to loop variable

In [ ]:
# Spatio-temporal variables

years = ['2019']

months = ['01', '02', '03',
          '04', '05', '06',
          '07', '08', '09',
          '10', '11', '12']

days = ['01', '02', '03'
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
        '13', '14', '15',
        '16', '17', '18',
        '19', '20', '21',
        '22', '23', '24',
        '25', '26', '27',
        '28', '29', '30',
        '31']

times = ['00:00', '01:00', '02:00',
         '03:00', '04:00', '05:00',
         '06:00', '07:00', '08:00',
         '09:00', '10:00', '11:00',
         '12:00', '13:00', '14:00',
         '15:00', '16:00', '17:00',
         '18:00', '19:00', '20:00',
         '21:00', '22:00', '23:00']

areas = ['north_america', ]  # 'australia']
area_dict = {'north_america': [70, -170, 25, -50],
             'australia': [-10, 110, -45, 155]}


# Pressure variables

pressure_variables = ['relative_humidity', 'vertical_velocity',
                      'u_component_of_wind', 'v_component_of_wind']

pressure_dict = {'relative_humidity': ['650', '750', '850'],
                 'u_component_of_wind': '250',
                 'v_component_of_wind': '250',
                 'vertical_velocity': '1'}


# Single level variables

single_lvl_variables = ['convective_inhibition', 'geopotential',
                        'surface_latent_heat_flux', 'surface_sensible_heat_flux',
                        '10m_u_component_of_wind', '10m_v_component_of_wind',
                        '10m_wind_gust_since_previous_post_processing', 'boundary_layer_height',
                        'convective_available_potential_energy']


# Fuel variables
fuel_variables = ['high_vegetation_cover', 'low_vegetation_cover',
                  'type_of_high_vegetation', 'type_of_low_vegetation']


def download_pressure_variables(years: list, months: list, days: list, times: list, areas: list,
                                area_dict: dict, pressure_variables: list, pressure_dict: dict):
    """
    Download pressure variables and copies data to Google Cloud Storage Bucket.

    Args:
        years (list): list of years (strings) to download
        months (list): list of months (strings) to download
        days (list): list of days (strings) to download
        times (list): list of times (strings) to download
        areas (list): list of areas to dowload
        area_dict (dict): dictionnary defining area extents using lat/lon coordinates
        pressure_variables (list): list of pressure variables to dowload
        pressure_dict (dict): dictionnary defining pressure levels to download for each pressure variable
    """

    for y in years:
        for v in pressure_variables:
            for a in areas:
                filename = v + '_' + y + '.nc'

                c.retrieve(
                    'reanalysis-era5-pressure-levels',
                    {
                        'product_type': 'reanalysis',
                        'format': 'netcdf',
                        'variable': v,
                        'pressure_level': pressure_dict[v],
                        'year': y,
                        'month': months,
                        'day': days,
                        'time': times,
                        'area': area_dict[a],
                    },
                    filename)

                move_command = 'gsutil cp ' + filename + \
                    ' gs://eu-aerosols-landing/ERA5/raw_data/' + a + '/'
                system(move_command)

                # delete file once it's been copied
                del_command = 'rm ' + filename
                system(del_command)


def download_single_lvl_variables(years: list, months: list, days: list, times: list,
                                  areas: list, area_dict: dict, single_lvl_variables: list):
    """
    Download single level variables and copies data to Google Cloud Storage Bucket.

    Args:
        years (list): list of years (strings) to download
        months (list): list of months (strings) to download
        days (list): list of days (strings) to download
        times (list): list of times (strings) to download
        areas (list): list of areas to download
        area_dict (dict): dictionnary defining area extents using lat/lon coordinates
        single_lvl_variables (list): list of single level variables to download
    """
    for y in years:
        for v in single_lvl_variables:
            for a in areas:

                filename = v + '_' + y + '.nc'

                c.retrieve(
                    'reanalysis-era5-single-levels',
                    {
                        'product_type': 'reanalysis',
                        'format': 'netcdf',
                        'variable': v,
                        'year': y,
                        'month': months,
                        'day': days,
                        'time': times,
                        'area': area_dict[a],
                    },
                    filename)

                move_command = 'gsutil cp ' + filename + \
                    ' gs://eu-aerosols-landing/ERA5/raw_data/' + a + '/'
                system(move_command)

                # delete file once it's been copied
                del_command = 'rm ' + filename
                system(del_command)

In [ ]:
#UID221000
#API Keyc3416410-1ee5-4997-b70d-890d6837e7a6

import cdsapi
from subprocess import call


def idmDownloader(task_url, folder_path, file_name):
    """
    IDM下载器
    :param task_url: 下载任务地址
    :param folder_path: 存放文件夹
    :param file_name: 文件名
    :return:
    """
    # IDM安装目录
    idm_engine = r"E:\app\Internet Download Manager\IDMan.exe"
    # 将任务添加至队列
    call([idm_engine, '/d', task_url, '/p', folder_path, '/f', file_name, '/a'])
    # 开始任务队列
    call([idm_engine, '/s'])


if __name__ == '__main__':
    c = cdsapi.Client()  # 创建用户

    # 数据信息字典
    dic = {
        'product_type': 'reanalysis',  # 产品类型
        'format': 'netcdf',  # 数据格式
        'variable': '2m_temperature',  # 变量名称
        'year': '',  # 年，设为空
        'month': [# 月
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],  
        'day': [# 日
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],  
        'time': [  # 小时
            '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'
        ]
    }

    for y in range(1980, 2022):  # 遍历年
        dic['year'] = str(y)

        r = c.retrieve('reanalysis-era5-single-levels', dic, )  # 文件下载器
        url = r.location  # 获取文件下载地址
        path = r'F:\t2m'  # 存放文件夹
        filename = str(y) + '.nc'  # 文件名
        idmDownloader(url, path, filename)  # 添加进IDM中下载

In [ ]:
  call([‘wget ’, task_url, ‘-O’, folder_path, file_name])

In [ ]:
call([‘wget ’, task_url, ‘ -O ’, folder_path, file_name])